In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxeswithdamagesanddents56.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxeswithdamagesanddents30.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/damagedfoodpackagingbox59.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxesthataredamaged234.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxesthataredamaged245.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/damagedfoodpackagingbox244.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxesthataredamaged18.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packagingboxesthataredamaged109.jpeg
/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged/packag

In [6]:
# Install required packages
!pip install ultralytics roboflow scikit-learn opencv-python torch

In [ ]:
from ultralytics import YOLO
import os
import torch
from sklearn.model_selection import train_test_split
import shutil

def create_yolo_dataset():
    base_dir = '/kaggle/working'
    os.makedirs(f'{base_dir}/dataset/images/train', exist_ok=True)
    os.makedirs(f'{base_dir}/dataset/images/val', exist_ok=True)
    os.makedirs(f'{base_dir}/dataset/labels/train', exist_ok=True)
    os.makedirs(f'{base_dir}/dataset/labels/val', exist_ok=True)

def prepare_data():
    damaged_dir = '/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/damaged'
    intact_dir = '/kaggle/input/damaged-and-intact-packages/damaged-and-intact-packages/intact'
    
    damaged_files = [f for f in os.listdir(damaged_dir) if f.endswith(('.jpg', '.jpeg'))]
    intact_files = [f for f in os.listdir(intact_dir) if f.endswith(('.jpg', '.jpeg'))]
    
    damaged_train, damaged_val = train_test_split(damaged_files, test_size=0.2, random_state=42)
    intact_train, intact_val = train_test_split(intact_files, test_size=0.2, random_state=42)
    
    base_dir = '/kaggle/working'
    
    for img_file in damaged_train:
        shutil.copy(os.path.join(damaged_dir, img_file), 
                   os.path.join(f'{base_dir}/dataset/images/train', img_file))
        create_label(img_file, 'train', 0)
    
    for img_file in intact_train:
        shutil.copy(os.path.join(intact_dir, img_file),
                   os.path.join(f'{base_dir}/dataset/images/train', img_file))
        create_label(img_file, 'train', 1)
    
    for img_file in damaged_val:
        shutil.copy(os.path.join(damaged_dir, img_file),
                   os.path.join(f'{base_dir}/dataset/images/val', img_file))
        create_label(img_file, 'val', 0)
    
    for img_file in intact_val:
        shutil.copy(os.path.join(intact_dir, img_file),
                   os.path.join(f'{base_dir}/dataset/images/val', img_file))
        create_label(img_file, 'val', 1)

def create_label(img_file, split, class_id):
    base_dir = '/kaggle/working'
    label_path = f'{base_dir}/dataset/labels/{split}/{os.path.splitext(img_file)[0]}.txt'
    with open(label_path, 'w') as f:
        f.write(f'{class_id} 0.5 0.5 1.0 1.0\n')

def create_yaml():
    yaml_content = """
train: /kaggle/working/dataset/images/train
val: /kaggle/working/dataset/images/val

nc: 2
names: ['damaged', 'intact']
    """
    
    with open('/kaggle/working/dataset.yaml', 'w') as f:
        f.write(yaml_content)

def train_model():
    model = YOLO('yolov8n.pt')
    results = model.train(
        data='/kaggle/working/dataset.yaml',
        epochs=50,
        imgsz=640,
        batch=16,
        workers=4,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    return model

if __name__ == "__main__":
    create_yolo_dataset()
    prepare_data()
    create_yaml()
    model = train_model()

Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=4, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

100%|██████████| 755k/755k [00:00<00:00, 3.75MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/dataset/labels/train... 433 images, 0 backgrounds, 0 corrupt: 100%|██████████| 433/433 [00:00<00:00, 943.40it/s]

train: New cache created: /kaggle/working/dataset/labels/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /kaggle/working/dataset/labels/val... 109 images, 0 backgrounds, 0 corrupt: 100%|██████████| 109/109 [00:00<00:00, 1012.93it/s]

val: New cache created: /kaggle/working/dataset/labels/val.cache


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G       1.01      2.945      1.556         54        640:  32%|███▏      | 9/28 [01:15<02:27,  7.75s/it]